In [4]:
!apt-get install -y swig
!pip install gymnasium[box2d]
!pip install matplotlib



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swig is already the newest version (4.0.2-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [5]:
import gymnasium as gym
import numpy as np

env = gym.make("LunarLander-v3", render_mode=None)

# parameters
target_x = 0.0
max_x_vel = 0.5
max_y_vel = -0.5
thrust_y = 0.5

def simple_lander_strategy(observation):

    pos_x, pos_y, vel_x, vel_y, angle, ang_vel, _, _ = observation

    # default
    action = 0
    final_action_weightage={"Nothing":0,"Left":0,"Thrust":0,"Right":0}

    # vertical control
    if vel_y < max_y_vel:
        action = 2
        final_action_weightage["Thrust"]+=2-(vel_y/max_y_vel)

    # horizontal control
    # left drift or pos
    if vel_x < -max_x_vel or (pos_x < target_x - 0.1 and pos_y > 0.1):
        action = 3 # move right
        final_action_weightage["Right"]+=2+(vel_x/max_x_vel)+(pos_x/target_x)

    # right drift or pos
    elif vel_x > max_x_vel or (pos_x > target_x + 0.1 and pos_y > 0.1):
        action = 1 # move left
        final_action_weightage["Left"]+=-2-(vel_x/max_x_vel)+(pos_x/target_x)

    # angle correction
    if pos_y < 0.2:
        if angle > 0.05:
            # to correct right, do left
            action = 1
            final_action_weightage["Left"]+=angle/0.05
        elif angle < -0.05:
            # to correct left, do right
            action = 3
            final_action_weightage["Right"]+=angle/0.05

    # stable then vertical control
    if pos_y < 0.2 and vel_y < -0.1:
        action = 2 # gentle thrust
        final_action_weightage["Thrust"]+=1

    return action

# action loop
observation, info = env.reset(seed=42)
run = True
total_reward = 0

while run:

    action = simple_lander_strategy(observation)

    observation, reward, terminated, truncated, info = env.step(action)
    total_reward += reward

    if terminated or truncated:
      observation, info = env.reset()
      run = False
      print(reward)
      if terminated and total_reward > 100:
        print("Landed")
      elif terminated:
        print("Crashed")
      else:
        print("Time Limit")

env.close()

-100
Crashed


/tmp/ipython-input-162071527.py:34: RuntimeWarning: divide by zero encountered in scalar divide
  final_action_weightage["Left"]+=-2-(vel_x/max_x_vel)+(pos_x/target_x)
